# Nicolas
---

## Importing stuff

In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

---
## Loading data

* **route**

In [2]:
routes = pd.read_csv('data/routes.dat', sep=',', encoding='utf-8')
routes.columns = ['Airline','AirlineID','SourceAirport','SourceAirportID','DestinationAirport','DestinationAirportID','Codeshare','Stops','Equipment']
routes.head()

,Airline,AirlineID,SourceAirport,SourceAirportID,DestinationAirport,DestinationAirportID,Codeshare,Stops,Equipment
0,2B,410,ASF,2966,KZN,2990,NaN,0,CR2
1,2B,410,ASF,2966,MRV,2962,NaN,0,CR2
2,2B,410,CEK,2968,KZN,2990,NaN,0,CR2
3,2B,410,CEK,2968,OVB,4078,NaN,0,CR2
4,2B,410,DME,4029,KZN,2990,NaN,0,CR2


* **airlines**

In [3]:
airlines = pd.read_csv('data/airlines.dat', sep=',', encoding='utf-8')
airlines.columns = ['AirlineID', 'Name', 'Alias',  'IATA', 'ICAO','Callsign','Country','Active']
airlines.head()

,AirlineID,Name,Alias,IATA,ICAO,Callsign,Country,Active
0,1,Private flight,\N,-,NaN,NaN,NaN,Y
1,2,135 Airways,\N,NaN,GNL,GENERAL,United States,N
2,3,1Time Airline,\N,1T,RNX,NEXTIME,South Africa,Y
3,4,2 Sqn No 1 Elementary Flying Training School,\N,NaN,WYT,NaN,United Kingdom,N
4,5,213 Flight Unit,\N,NaN,TFU,NaN,Russia,N


* **airports**

In [4]:
airports = pd.read_csv('data/airports.dat', sep=',', encoding='utf-8')
airports.columns = ['AirportID', 'Name', 'City', 'Country', 'IATA', 'ICAO','Latitude','Longitude','Altitude', 'Timezone','DST', 'TzdatabaseTimeZone', 'Type', 'Source']
airports.head()

,AirportID,Name,City,Country,IATA,ICAO,Latitude,Longitude,Altitude,Timezone,DST,TzdatabaseTimeZone,Type,Source
0,2,Madang Airport,Madang,Papua New Guinea,MAG,AYMD,-5.207080,145.789001,20,10,U,Pacific/Port_Moresby,airport,OurAirports
1,3,Mount Hagen Kagamuga Airport,Mount Hagen,Papua New Guinea,HGU,AYMH,-5.826790,144.296005,5388,10,U,Pacific/Port_Moresby,airport,OurAirports
2,4,Nadzab Airport,Nadzab,Papua New Guinea,LAE,AYNZ,-6.569803,146.725977,239,10,U,Pacific/Port_Moresby,airport,OurAirports
3,5,Port Moresby Jacksons International Airport,Port Moresby,Papua New Guinea,POM,AYPY,-9.443380,147.220001,146,10,U,Pacific/Port_Moresby,airport,OurAirports
4,6,Wewak International Airport,Wewak,Papua New Guinea,WWK,AYWK,-3.583830,143.669006,19,10,U,Pacific/Port_Moresby,airport,OurAirports


* **alliances**

In [5]:
alliances = pd.read_csv('data/alliances.dat', sep=',', encoding='utf-8')
alliances.head()

,Alliance,Airline
0,Star Alliance,Adria Airways
1,Star Alliance,Aegean Airlines
2,Star Alliance,Air Canada
3,Star Alliance,Air China
4,Star Alliance,Air New Zealand


In [6]:
alliances.shape

(60, 2)

All the airlines from `alliances.dat` now have a correspondance in the airlines data base with the rigth name.

---
## Creating some array

Merging the original Airline ID with it Alliance

In [56]:
allianceAirlines  = pd.merge(alliances, airlines, left_on='Airline', right_on='Name', how = 'left')
allianceAirlines[allianceAirlines['AirlineID'].isnull()]
allianceAirlines.AirlineID = allianceAirlines.AirlineID.astype('int')
allianceAirlines = allianceAirlines[["Alliance", "Name", "AirlineID"]].set_index("AirlineID")
allianceAirlines.head()

,Alliance,Name
AirlineID,,
83,Star Alliance,Adria Airways
96,Star Alliance,Aegean Airlines
330,Star Alliance,Air Canada
751,Star Alliance,Air China
345,Star Alliance,Air New Zealand


In [57]:
allianceID = pd.DataFrame({"Star Alliance":[2],"SkyTeam":[3], "One World":[4]}).transpose()
allianceID = allianceID.rename(columns={0:'AllianceID'})
allianceID

,AllianceID
Star Alliance,2
SkyTeam,3
One World,4


In [58]:
allianceAirlines = allianceAirlines.join(allianceID, on="Alliance", how="right")
allianceAirlines.head()

,Alliance,Name,AllianceID
AirlineID,,,
83,Star Alliance,Adria Airways,2
96,Star Alliance,Aegean Airlines,2
330,Star Alliance,Air Canada,2
751,Star Alliance,Air China,2
345,Star Alliance,Air New Zealand,2


(re)Creating the airport ID (from Milestone 1)

In [59]:
# import of source and destination airport
source_airports = routes[['SourceAirport']]
source_airports = source_airports.rename(columns={'SourceAirport':'Airport'})

dest_airports = routes[['DestinationAirport']]
dest_airports = dest_airports.rename(columns={'DestinationAirport':'Airport'})

# creation of a dataframe with all airport and airport_idx 
# (we use airport_idx insteed of airportID because some airports have no airportID)
airportsID = pd.concat([source_airports,dest_airports])
airportsID = airportsID.drop_duplicates()
airportsID.reset_index(inplace=True)
airportsID = airportsID.drop(columns=['index'])
airportsID.reset_index(inplace=True)
airportsID = airportsID.set_index('Airport')
airportsID = airportsID.rename(columns={'index':'airportsID'})

In [60]:
airportsID.head()

,airportsID
Airport,
ASF,0
CEK,1
DME,2
EGO,3
GYD,4


Creating the airlines ID

In [97]:
airlinesID = routes[['AirlineID']]
airlinesID = airlinesID.drop_duplicates()
airlinesID = airlinesID.replace('\\N',np.NaN).dropna()
airlinesID.AirlineID= airlinesID.AirlineID.astype('int')
airlinesID.reset_index(inplace=True)
airlinesID = airlinesID.drop(columns=['index'])
airlinesID = airlinesID.join(allianceAirlines, how = 'outer')
airlinesID

,AirlineID,Alliance,Name,AllianceID
0,410.0,NaN,NaN,NaN
1,1654.0,NaN,NaN,NaN
2,8359.0,NaN,NaN,NaN
3,470.0,NaN,NaN,NaN
4,1338.0,NaN,NaN,NaN
5,2750.0,NaN,NaN,NaN
6,3652.0,NaN,NaN,NaN
7,146.0,NaN,NaN,NaN
8,897.0,NaN,NaN,NaN
9,1729.0,NaN,NaN,NaN


In [98]:
airlinesID[airlinesID.AllianceID.notnull()]

,AirlineID,Alliance,Name,AllianceID
24,2524.0,One World,American Airlines,4.0
28,341.0,Star Alliance,Asiana Airlines,2.0
83,96.0,Star Alliance,Adria Airways,2.0
90,43.0,SkyTeam,Air Europa,3.0
96,218.0,Star Alliance,Aegean Airlines,2.0
130,5982.0,SkyTeam,Aeroflot Russian Airlines,3.0
137,1889.0,SkyTeam,Air France,3.0
214,2688.0,One World,Air Berlin,4.0
321,2143.0,SkyTeam,AeroMéxico,3.0
324,3498.0,Star Alliance,All Nippon Airways,2.0


In [88]:
airlinesID.shape

(593, 4)